# Calico

## Setup Minikube

In [171]:
import subprocess
import yaml
from typing import List
def get_checks(gh_actions_file: str):
    """Get checks from a GitHub Actions file"""
    with open(gh_actions_file) as f:
        actions = yaml.safe_load(f)
    steps = actions['jobs']['scavenger']['steps']
    checks = [step for step in steps if 'Check ' in step['name']]
    return checks

def run_check(check):
    # cmd = f"cd /workspaces/intro-to-kube\n{check['run']}"
    cmd = check['run']
    subprocess.run(
        cmd, 
        capture_output=False, 
        shell=True, 
        executable='/bin/bash',
        cwd = '/workspaces/intro-to-kube'
    )

def run_checks(checks: List):
    """Run checks"""
    for check in checks:
        run_check(check)
        
checks = get_checks('/workspaces/intro-to-kube/.github/workflows/calico.yml')

In [172]:
!minikube start --network-plugin=cni --cni=calico

😄  minikube v1.31.2 on Ubuntu 20.04 (docker/amd64)
✨  Automatically selected the docker driver. Other choices: none, ssh
❗  With --network-plugin=cni, you will need to provide your own CNI. See --cni flag as a user-friendly alternative
📌  Using Docker driver with root privileges
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🔥  Creating docker container (CPUs=2, Memory=2200MB) ...
🐳  Preparing Kubernetes v1.27.4 on Docker 24.0.4 ...
    ▪ Generating certificates and keys ...
    ▪ Booting up control plane ...
    ▪ Configuring RBAC rules ...
🔗  Configuring Calico (Container Networking Interface) ...
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


## Create Pods

In [173]:
%%bash
# Create nginx pod in namespace a
kubectl create ns namespace-a
kubectl config set-context --current --namespace=namespace-a
kubectl create -f ./k8s/nginx-pod.yml --namespace=namespace-a

# Create nginx pod in namespace b
kubectl create ns namespace-b
kubectl config set-context --current --namespace=namespace-b
kubectl create -f ./k8s/nginx-pod.yml --namespace=namespace-b

# Create some postgres pod inside namespace c
kubectl create ns namespace-c
kubectl config set-context --current --namespace=namespace-c
kubectl create -f ./k8s/postgres-pod.yml --namespace=namespace-c

namespace/namespace-a created
Context "minikube" modified.
pod/nginx created
namespace/namespace-b created
Context "minikube" modified.
pod/nginx created
namespace/namespace-c created
Context "minikube" modified.
pod/postgres created


In [208]:
!kubectl get pods -o wide -A

NAMESPACE     NAME                                       READY   STATUS    RESTARTS        AGE     IP              NODE       NOMINATED NODE   READINESS GATES
kube-system   calico-kube-controllers-85578c44bf-lmn6n   1/1     Running   4 (8m52s ago)   9m26s   10.244.120.68   minikube   <none>           <none>
kube-system   calico-node-rpzzj                          1/1     Running   0               9m27s   192.168.49.2    minikube   <none>           <none>
kube-system   coredns-5d78c9869d-k6244                   1/1     Running   3 (8m41s ago)   9m26s   10.244.120.67   minikube   <none>           <none>
kube-system   etcd-minikube                              1/1     Running   0               9m40s   192.168.49.2    minikube   <none>           <none>
kube-system   kube-apiserver-minikube                    1/1     Running   0               9m39s   192.168.49.2    minikube   <none>           <none>
kube-system   kube-controller-manager-minikube           1/1     Running   0               

In [210]:
%%bash
kubectl config use-context minikube
IP_POD_A=$(kubectl get pod nginx -n namespace-b -o jsonpath='{.status.podIP}')
kubectl exec -it nginx -n namespace-a -- sh -c "curl -s $IP_POD_A:80" 2>/dev/null

Switched to context "minikube".


Error while terminating subprocess (pid=256013): 


I1010 23:50:10.062227  255104 log.go:194] (0xc0001651e0) (0xc0003bd180) Stream removed, broadcasting: 3
I1010 23:50:10.062267  255104 log.go:194] (0xc0001651e0) (0xc0005005a0) Stream removed, broadcasting: 5
I1010 23:50:10.062282  255104 log.go:194] (0xc0001651e0) Data frame received for 1
I1010 23:50:10.062287  255104 log.go:194] (0xc000500500) (1) Data frame handling
I1010 23:50:10.062290  255104 log.go:194] (0xc000500500) (1) Data frame sent
I1010 23:50:10.062293  255104 log.go:194] (0xc0001651e0) (0xc000500500) Stream removed, broadcasting: 1
I1010 23:50:10.062918  255104 log.go:194] (0xc0001651e0) (0xc0003bd220) Stream removed, broadcasting: 7
I1010 23:50:10.062937  255104 log.go:194] (0xc0001651e0) (0xc000500500) Stream removed, broadcasting: 1
I1010 23:50:10.062944  255104 log.go:194] (0xc0001651e0) (0xc0003bd180) Stream removed, broadcasting: 3
I1010 23:50:10.062949  255104 log.go:194] (0xc0001651e0) (0xc0005005a0) Stream removed, broadcasting: 5
I1010 23:50:10.062955  255104 l

In [192]:
# Install curl in postgres pod
!kubectl exec -it postgres -n namespace-c -- su -c "apt-get -y update && apt-get -y install curl" root 2>/dev/null

Get:1 http://deb.debian.org/debian bookworm InRelease [151 kB]
Get:2 http://deb.debian.org/debian bookworm-updates InRelease [52.1 kB]
Get:3 http://deb.debian.org/debian-security bookworm-security InRelease [48.0 kB]
Get:4 http://apt.postgresql.org/pub/repos/apt bookworm-pgdg InRelease [123 kB]
Get:5 http://deb.debian.org/debian bookworm/main amd64 Packages [8,780 kB]
Get:6 http://deb.debian.org/debian bookworm-updates/main amd64 Packages [6,408 B]
Get:7 http://deb.debian.org/debian-security bookworm-security/main amd64 Packages [79.4 kB]
Get:8 http://apt.postgresql.org/pub/repos/apt bookworm-pgdg/main amd64 Packages [296 kB]
Fetched 9,536 kB in 1s (6,646 kB/s)                      
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  ca-certificates libbrotli1 libcurl4 libnghttp2-14 libpsl5 librtmp1 libssh2-1
  publicsuffix
The following NEW packages will be

In [193]:
# Check curl version in postgres pod
!kubectl exec -it postgres -n namespace-c -- sh -c "curl --version" 2>/dev/null

curl 7.88.1 (x86_64-pc-linux-gnu) libcurl/7.88.1 OpenSSL/3.0.9 zlib/1.2.13 brotli/1.0.9 zstd/1.5.4 libidn2/2.3.3 libpsl/0.21.2 (+libidn2/2.3.3) libssh2/1.10.0 nghttp2/1.52.0 librtmp/2.3 OpenLDAP/2.5.13
Release-Date: 2023-02-20
Protocols: dict file ftp ftps gopher gophers http https imap imaps ldap ldaps mqtt pop3 pop3s rtmp rtsp scp sftp smb smbs smtp smtps telnet tftp
Features: alt-svc AsynchDNS brotli GSS-API HSTS HTTP2 HTTPS-proxy IDN IPv6 Kerberos Largefile libz NTLM NTLM_WB PSL SPNEGO SSL threadsafe TLS-SRP UnixSockets zstd


## Set Networking Policies

![networking](https://uplimit.com/_next/image?url=https%3A%2F%2Fuplimit-ugc.com%2Fstatic%2Fcourse%2Fkubernetes-managing-containers-at-scale%2Fassets%2Fclm9w6ld001hk12ad3lx6dequ%2Fdiagram.png&w=750&q=75)

### Namespace A

In [211]:
%%writefile network-policy-namespace-a.yml
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: network-policy-for-namespace-a
  namespace: namespace-a
spec:
  ingress:
  - from:
    - namespaceSelector:
        matchLabels:
          name: namespace-b
    ports:
      - protocol: TCP
        port: 80
  egress:
  - to:
    - namespaceSelector:
        matchLabels:
          name: namespace-c
    ports:
      - protocol: TCP
        port: 5432


Overwriting network-policy-namespace-a.yml


In [212]:
!kubectl apply -f network-policy-namespace-a.yml

networkpolicy.networking.k8s.io/network-policy-for-namespace-a unchanged


In [213]:
!kubectl describe networkpolicy network-policy-for-namespace-a -n namespace-a

Name:         network-policy-for-namespace-a
Namespace:    namespace-a
Created on:   2023-10-10 23:42:09 +0000 UTC
Labels:       <none>
Annotations:  <none>
Spec:
  PodSelector:     <none> (Allowing the specific traffic to all pods in this namespace)
  Allowing ingress traffic:
    To Port: 80/TCP
    From:
      NamespaceSelector: name=namespace-b
  Allowing egress traffic:
    To Port: 5432/TCP
    To:
      NamespaceSelector: name=namespace-c
  Policy Types: Ingress, Egress


In [201]:
run_check(checks[0])

networkpolicy.networking.k8s.io/network-policy-for-namespace-a unchanged
❌ Cannot ping pods in namespace-b
❌ Cannot ping pods in namespace-c


I1010 23:45:16.748400  249742 log.go:194] (0xc0000e4420) (0xc00051d540) Create stream
I1010 23:45:16.748903  249742 log.go:194] (0xc0000e4420) (0xc00051d540) Stream added, broadcasting: 1
I1010 23:45:16.752284  249742 log.go:194] (0xc0000e4420) Reply frame received for 1
I1010 23:45:16.752326  249742 log.go:194] (0xc0000e4420) (0xc00062fea0) Create stream
I1010 23:45:16.752499  249742 log.go:194] (0xc0000e4420) (0xc00062fea0) Stream added, broadcasting: 3
I1010 23:45:16.753335  249742 log.go:194] (0xc0000e4420) Reply frame received for 3
I1010 23:45:16.753353  249742 log.go:194] (0xc0000e4420) (0xc00051d5e0) Create stream
I1010 23:45:16.753360  249742 log.go:194] (0xc0000e4420) (0xc00051d5e0) Stream added, broadcasting: 5
I1010 23:45:16.754174  249742 log.go:194] (0xc0000e4420) Reply frame received for 5
I1010 23:45:16.848706  249742 log.go:194] (0xc0000e4420) Data frame received for 5
I1010 23:45:16.848725  249742 log.go:194] (0xc00051d5e0) (5) Data frame handling
I1010 23:45:16.84873

### Namespace B

In [203]:
%%writefile network-policy-namespace-b.yml
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: network-policy-for-namespace-b
  namespace: namespace-b
spec:
  ingress:
  - from:
    - namespaceSelector:
        matchLabels:
          name: namespace-a
  egress:
  - to:
    - namespaceSelector:
        matchLabels:
          name: namespace-a

Overwriting network-policy-namespace-b.yml


In [204]:
run_check(checks[1])

networkpolicy.networking.k8s.io/network-policy-for-namespace-b created
❌ Cannot ping pods in namespace-a


I1010 23:46:32.143011  252186 log.go:194] (0xc0001d4000) (0xc0003c2000) Create stream
I1010 23:46:32.143674  252186 log.go:194] (0xc0001d4000) (0xc0003c2000) Stream added, broadcasting: 1
I1010 23:46:32.144984  252186 log.go:194] (0xc0001d4000) Reply frame received for 1
I1010 23:46:32.145010  252186 log.go:194] (0xc0001d4000) (0xc000674960) Create stream
I1010 23:46:32.145017  252186 log.go:194] (0xc0001d4000) (0xc000674960) Stream added, broadcasting: 3
I1010 23:46:32.145847  252186 log.go:194] (0xc0001d4000) Reply frame received for 3
I1010 23:46:32.146020  252186 log.go:194] (0xc0001d4000) (0xc0006757c0) Create stream
I1010 23:46:32.146140  252186 log.go:194] (0xc0001d4000) (0xc0006757c0) Stream added, broadcasting: 5
I1010 23:46:32.147158  252186 log.go:194] (0xc0001d4000) Reply frame received for 5
I1010 23:46:32.211907  252186 log.go:194] (0xc0001d4000) Data frame received for 5
I1010 23:46:32.212299  252186 log.go:194] (0xc0006757c0) (5) Data frame handling
I1010 23:46:32.21242

KeyboardInterrupt: 

### Namespace C

In [53]:
%%writefile network-policy-namespace-c.yml
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: network-policy-for-namespace-c
  namespace: namespace-c
spec:
  ingress:
  - from:
    - namespaceSelector:
        matchLabels:
          name: namespace-a

Overwriting network-policy-namespace-c.yml


In [ ]:
run_check(checks[2])

## Teardown

In [170]:
!minikube stop
!minikube delete

✋  Stopping node "minikube"  ...
🛑  Powering off "minikube" via SSH ...
🛑  1 node stopped.
🔥  Deleting "minikube" in docker ...
🔥  Deleting container "minikube" ...
🔥  Removing /home/codespace/.minikube/machines/minikube ...
💀  Removed all traces of the "minikube" cluster.
